# Langchain으로 RDF 형식 데이터 활용하기
---
- 2024-02-13

In [1]:
import os
import dotenv
dotenv.load_dotenv()

# 코랩 환경에서 실행하고 있다면 os.getenv("OPENAI_API_KEY") 를 지우고 본인의 api key를 입력하면 된다  
API_KEY = os.getenv("OPENAI_API_KEY") # .env 파일에 있는 OPENAI_API_KEY를 가져온다

from langchain.chains import GraphSparqlQAChain
from langchain_community.graphs import RdfGraph
from langchain_openai import ChatOpenAI

## 1. 기본 사용법

### 1-1. 그래프 로드하기

In [19]:
graph = RdfGraph(
    source_file="test.ttl",
    standard="rdf",
    local_copy="test.ttl",  # 그래프에 대한 변경사항이 존재할 경우 로컬에 저장토록 한다.
)

업데이트 된 그래프를 새로고침하는 코드이다.

In [3]:
graph.load_schema()

스키마를 직접 확인하는 코드이다

In [33]:
graph.get_schema

'In the following, each IRI is followed by the local name and optionally its description in parentheses. \nThe RDF graph supports the following node types:\n<http://xmlns.com/foaf/0.1/PersonalProfileDocument> (PersonalProfileDocument, None), <http://www.w3.org/2000/10/swap/pim/contact#Male> (Male, None), <http://xmlns.com/foaf/0.1/Person> (Person, None), <http://www.w3.org/2006/vcard/ns#Work> (Work, None), <http://www.w3.org/ns/auth/cert#RSAPublicKey> (RSAPublicKey, None)\nThe RDF graph supports the following relationships:\n<http://xmlns.com/foaf/0.1/maker> (maker, None), <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> (type, None), <http://creativecommons.org/ns#license> (license, None), <http://purl.org/dc/elements/1.1/title> (title, None), <http://www.w3.org/2006/vcard/ns#hasAddress> (hasAddress, None), <http://xmlns.com/foaf/0.1/based_near> (based_near, None), <http://www.w3.org/ns/solid/terms#editableProfile> (editableProfile, None), <http://xmlns.com/foaf/0.1/member> (member, 

### 1-2. 그래프 내용을 기반한 질의

In [5]:
chain = GraphSparqlQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)
chain.run("What is Tim Berners-Lee's work homepage?")

c:\Users\user\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphSparqlQAChain chain...
Identified intent:
SELECT
Generated SPARQL:
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT ?homepage
WHERE {
    ?person foaf:name "Tim Berners-Lee" .
    ?person foaf:workplaceHomepage ?homepage .
}
Full Context:
[]

> Finished chain.


"Tim Berners-Lee's work homepage is http://www.w3.org/People/Berners-Lee/."

### 1-3. 그래프 내용 변경하기
- 자연어 쿼리를 통해 그래프의 관계, 속성 등을 변경할 수 있다.

In [31]:
chain.run(
    "Save that the person with the name 'Timothy Berners-Lee' has a work homepage at 'http://www.w3.org/foo/bar/dd'"
)



> Entering new GraphSparqlQAChain chain...
Identified intent:
UPDATE
Generated SPARQL:
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
INSERT {
    ?person foaf:workplaceHomepage <http://www.w3.org/foo/bar/dd> .
}
WHERE {
    ?person foaf:name "Timothy Berners-Lee" .
}

> Finished chain.


'Successfully inserted triples into the graph.'

In [29]:
graph.load_schema()

In [32]:
query = (
    """PREFIX foaf: <http://xmlns.com/foaf/0.1/>\n"""
    """SELECT ?hp\n"""
    """WHERE {\n"""
    """    ?person foaf:name "Timothy Berners-Lee" . \n"""
    """    ?person foaf:workplaceHomepage ?hp .\n"""
    """}"""
)
graph.query(query) # 그래프에 대해 바로 쿼리할 수 있다.

[(rdflib.term.URIRef('http://www.w3.org/foo/bar/'),),
 (rdflib.term.URIRef('https://www.w3.org/'),)]